In [ ]:
import json

import gradio as gr
import pandas as pd

from library.types import *
from library.utils import *

In [ ]:
with open("artifacts/feedbacks/labeled.json", "r") as file:
    feedbacks = json.load(file)

In [ ]:
# TODO: Add inputs to label issues, suggestions and reviews
with gr.Blocks() as demo:
    def find_unlabeled(): 
        for index, feedback in enumerate(feedbacks):
            if any([feedback.get(label) is None for label in ["language", "translation", "issues"]]):
                return index
        return 0

    index = gr.State(find_unlabeled())

    with gr.Row():
        with gr.Column():
            customer = gr.Text(value=f"({index.value+1}/{len(feedbacks)}) {feedbacks[index.value]['customer']}", label="customer")
            content = gr.TextArea(value=feedbacks[index.value]["content"], label="content", show_copy_button=True)

        with gr.Column():
            language = gr.Dropdown(value=feedbacks[index.value].get("language"), label="language", choices=["ENGLISH", "SPANISH", "FRENCH", "ITALIAN", "GERMAN", "PORTUGUESE", "UNKNOWN"])
            translation = gr.TextArea(value=feedbacks[index.value].get("translation"), label="translation")

    issues = gr.DataFrame(
        value=pd.DataFrame.from_records(feedbacks[index.value].get("issues", []), columns=["title", "description"]),
        label="issues", headers=["title", "description"], col_count=(2, "fixed"), row_count=(0, "dynamic"))

    render = lambda index, feedback: {
        customer: f"({index+1}/{len(feedbacks)}) {feedback['customer']}",
        content: feedback["content"],
        language: feedback.get("language"),
        translation: feedback.get("translation"),
        issues: pd.DataFrame.from_records(feedback.get("issues", []), columns=["title", "description"]),
    }

    @gr.Button("Find Unlabeled").click(outputs=[index, customer, content, language, translation, issues])
    def find():
        new_index = find_unlabeled()
        return {
            index: new_index,
            **render(new_index, feedbacks[new_index])
            }

    with gr.Row():        
        @gr.Button("<< Prev").click(inputs=index, outputs=[index, customer, content, language, translation, issues])
        def prev(old_index):
            new_index = (old_index - 1) % len(feedbacks)
            return {
                index: new_index,
                **render(new_index, feedbacks[new_index])
            }

        @gr.Button("Next >>").click(inputs=index, outputs=[index, customer, content, language, translation, issues])
        def next(old_index):
            new_index = (old_index + 1) % len(feedbacks)
            return {
                index: new_index,
                **render(new_index, feedbacks[new_index])
            }

    @gr.Button("Save", variant="primary").click(inputs=[index, language, translation, issues])
    def save(index, language, translation, issues):
        feedbacks[index]["language"] = language
        feedbacks[index]["translation"] = translation
        feedbacks[index]["issues"] = list(filter(lambda i: any(i.values()), issues.to_dict("records")))

        with open("artifacts/feedbacks/labeled.json", "w") as file:
            json.dump(feedbacks, file, indent=2)

    demo.launch(inline=False, inbrowser=True)